In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SequentialFeatureSelector
import pandas as pd

toyota = toyota_cut.copy()

scaler = MinMaxScaler()
toyota = scaler.fit_transform(toyota)
toyota = pd.DataFrame(toyota)
toyota.columns = toyota_cut.columns

X = toyota.drop(columns=["Price"], axis=1)
y = toyota["Price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

sfs = SequentialFeatureSelector(LinearRegression(), n_features_to_select=10, direction="forward")
sfs.fit(X_train, y_train)
sfs.get_support()

print(X_train.columns[sfs.get_support()])

In [ ]:
sfsb = SequentialFeatureSelector(LinearRegression(), n_features_to_select=10, direction="backward")
sfsb.fit(X_train, y_train)
sfsb.get_support()

print(X_train.columns[sfs.get_support()])